In [12]:
import cv2
import numpy as np
from scipy.signal import convolve2d

def gaussian_psf(size, sigma):
    kernel = cv2.getGaussianKernel(size, sigma)
    return np.outer(kernel, kernel)

def wiener_filter(degraded_image, psf, snr):
    # Perform Fourier Transform on the degraded image and PSF
    degraded_image_fft = np.fft.fft2(degraded_image)
    psf_fft = np.fft.fft2(psf, s=degraded_image.shape)

    # Calculate the power spectral density (PSD) of the degraded image
    psd_degraded = np.abs(degraded_image_fft) ** 2 / degraded_image.size

    # Estimate the PSD of the original image using the Wiener filter formula
    psd_restored = psd_degraded / (snr + psd_degraded)

    # Apply the Wiener filter in the frequency domain
    restored_image_fft = np.conj(psf_fft) * degraded_image_fft / (np.abs(psf_fft) ** 2 + psd_restored)

    # Inverse Fourier Transform to get the restored image
    restored_image = np.fft.ifft2(restored_image_fft).real

    # Clip to ensure pixel values are in the valid range [0, 255]
    restored_image = np.clip(restored_image, 0, 255).astype(np.uint8)

    return restored_image

# Load the degraded image
degraded_image = cv2.imread('Lincoln.jpg', cv2.IMREAD_GRAYSCALE)

# Define the parameters for the Gaussian PSF
psf_size = 15  # Adjust the size as needed
psf_sigma = 2  # Adjust the sigma as needed

# Generate the Gaussian PSF
psf = gaussian_psf(psf_size, psf_sigma)

# Set the signal-to-noise ratio (SNR)
snr = 20  # Adjust as needed

# Apply the Wiener filter
restored_image = wiener_filter(degraded_image, psf, snr)

# Save or display the restored image
cv2.imwrite('restored_image.jpg', restored_image)
cv2.imshow('Restored Image', restored_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
